In [52]:
import os
import json
import traceback
import pandas as pd

In [1]:
import langchain
%pip install langchain_google_genai
%pip install PyPDF2

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [54]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [55]:
from dotenv import load_dotenv

load_dotenv()

True

In [56]:
KEY=os.getenv("GOOGLE_API_KEY")


In [57]:
llm=ChatGoogleGenerativeAI(google_api_key=KEY,model="gemini-2.5-flash",temperature=0.5, max_output_tokens=1500)

In [58]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
import PyPDF2

In [ ]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here", 
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here", 
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here", 
        },
        "correct": "correct answer",
    },
}

In [60]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz of {number} multiple choice questions for {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like RESPONSE_JSON below and use it as a guide.\
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [61]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text","number","subject","tone","response_json"],
    template=TEMPLATE
)

In [62]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz",verbose=True)

In [63]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity
and if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits
the students ability.
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [64]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject","quiz"], template=TEMPLATE)

In [65]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [66]:
generate_evaluate_chains=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text","number","subject","tone","response_json"],
                                         output_variables=["quiz","review"], verbose=True,)


In [67]:
file_path=r"E:\AI\data.txt"

In [68]:
with open(file_path,'r') as file:
    TEXT = file.read()

In [69]:
print(TEXT)

India, officially the Republic of India,[j][20] is a country in South Asia. It is the seventh-largest country by area; the most populous country since 2023;[21] and, since its independence in 1947, the world's most populous democracy.[22][23][24] Bounded by the Indian Ocean on the south, the Arabian Sea on the southwest, and the Bay of Bengal on the southeast, it shares land borders with Pakistan to the west;[k] China, Nepal, and Bhutan to the north; and Bangladesh and Myanmar to the east. In the Indian Ocean, India is near Sri Lanka and the Maldives; its Andaman and Nicobar Islands share a maritime border with Myanmar, Thailand, and Indonesia.

Modern humans arrived on the Indian subcontinent from Africa no later than 55,000 years ago.[26][27][28] Their long occupation, predominantly in isolation as hunter-gatherers, has made the region highly diverse.[29] Settled life emerged on the subcontinent in the western margins of the Indus river basin 9,000 years ago, evolving gradually into 

In [70]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [71]:
NUMBER=5
SUBJECT="India"
TONE="simple"

In [72]:

response=generate_evaluate_chains(
        {
            "text":TEXT,
            "number": NUMBER,
            "subject": SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
)



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:India, officially the Republic of India,[j][20] is a country in South Asia. It is the seventh-largest country by area; the most populous country since 2023;[21] and, since its independence in 1947, the world's most populous democracy.[22][23][24] Bounded by the Indian Ocean on the south, the Arabian Sea on the southwest, and the Bay of Bengal on the southeast, it shares land borders with Pakistan to the west;[k] China, Nepal, and Bhutan to the north; and Bangladesh and Myanmar to the east. In the Indian Ocean, India is near Sri Lanka and the Maldives; its Andaman and Nicobar Islands share a maritime border with Myanmar, Thailand, and Indonesia.

Modern humans arrived on the Indian subcontinent from Africa no later than 55,000 years ago.[26][27][28] Their long occupation, predominantly in isolation as hunter-gatherers, has made the region highly diverse.[29] Settled life emerged o

In [73]:
quiz=response.get("quiz")

In [74]:
import ast

# remove code fences
import re
cleaned = re.sub(r"^```json\n", "", quiz)
cleaned = re.sub(r"\n```$", "", cleaned)

data = ast.literal_eval(cleaned)   # handles single quotes, dict-style
print(data.keys())


dict_keys(['1', '2', '3', '4', '5'])


In [75]:
quiz=data

In [76]:
quiz_table_data=[]
for key,value in quiz.items():
    mcq= value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
        ]
    )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq,"Choices": options, "Correct": correct})

In [77]:
quiz_table_data

[{'MCQ': 'What is the official name of India, as mentioned in the text?',
  'Choices': 'a: Hindustan | b: Bharat | c: The Republic of India | d: Indian Subcontinent',
  'Correct': 'c'},
 {'MCQ': 'When did settled life emerge on the Indian subcontinent, which later led to the Indus Valley Civilisation?',
  'Choices': 'a: 55,000 years ago | b: 9,000 years ago | c: 1200 BCE | d: 400 BCE',
  'Correct': 'b'},
 {'MCQ': 'Which empire in South India created a long-lasting composite Hindu culture during the medieval era?',
  'Choices': 'a: Maurya Empire | b: Gupta Empire | c: Delhi Sultanate | d: Vijayanagara Empire',
  'Correct': 'd'},
 {'MCQ': 'In which year did British Crown rule begin in India?',
  'Choices': 'a: 1947 | b: 1858 | c: 1920 | d: 1757',
  'Correct': 'b'},
 {'MCQ': 'Since its independence in 1947, what is India known for being, according to the text?',
  'Choices': "a: The seventh-largest country by area | b: The world's most populous democracy | c: The country with the oldest c

In [78]:
quiz=pd.DataFrame(quiz_table_data)

In [79]:
quiz.to_csv("machinelearning.csv",index=False)